<a href="https://colab.research.google.com/github/vdragon95/lol_project/blob/develop/LOL03_match_data_and_ingame_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd

'/content'

In [ ]:
import requests
import pandas as pd
import numpy as np
import time

api_key = 'RGAPI-8c4988dd-493d-4518-a2af-fc916fe361aa'

start_point = 0 # 시작할 숫자 입력
end_point = 10000 # 끝낼 숫자 입력

tier = 'grandmaster' # master, challenger

In [ ]:
# <Match data>

# match_info_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/' + tier + '/' + 
#                 tier + '_' + division + '_02match_info_data.csv', encoding = 'cp949')
match_info_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/' + tier + '/' + \
                tier + '_02match_info_data.csv', encoding = 'cp949')
print(len(match_info_df)) # → 해당 tier 매치 수 (만 단위 씩 끊어서 저장 할 것)

66856


In [ ]:
# queue = 420  'RANKED_SOLO_5x5' 이 모드만 취급하기 위함


drop_idx = match_info_df[match_info_df['queue'] != 420].index
match_info_df = match_info_df.drop(drop_idx)
match_info_df = match_info_df.reset_index(drop=True)

match_info_df.to_csv('/content/drive/MyDrive/Colab Notebooks/' + tier + '/' + \
                tier + '_02match_info_queue_revised_data.csv',index=False, encoding = 'cp949')
print('revised version of match_info_data File successfully saved')
print('you may load the rest of datas:',len(match_info_df))

revised version of match_info_data File successfully saved
you may load the rest of datas: 53810


In [ ]:
match = []

# for i in range(len(match_info_df)):    
for i in range(start_point, end_point):    
    
    api_url='https://kr.api.riotgames.com/lol/match/v4/matches/' + str(match_info_df['gameId'].iloc[i]) + '?api_key=' + api_key
    r = requests.get(api_url)

    if r.status_code == 200: # response가 정상이면 바로 맨 밑으로 이동하여 정상적으로 코드 실행
        pass

    elif r.status_code == 429:
        print('api cost full : infinite loop start')
        print('loop location : ',i)
        start_time = time.time()

        while True: # 429error가 끝날 때까지 무한 루프
            if r.status_code == 429:

                print('try 10 second wait time')
                time.sleep(10)

                r = requests.get(api_url)
                print(r.status_code)

            elif r.status_code == 200: #다시 response 200이면 loop escape
                print('total wait time : ', time.time() - start_time)
                print('recovery api cost')
                break

    elif r.status_code == 503: # 잠시 서비스를 이용하지 못하는 에러
        print('service available error')
        start_time = time.time()

        while True:
            if r.status_code == 503 or r.status_code == 429:

                print('try 10 second wait time')
                time.sleep(10)

                r = requests.get(api_url)
                print(r.status_code)

            elif r.status_code == 200: # 똑같이 response가 정상이면 loop escape
                print('total error wait time : ', time.time() - start_time)
                print('recovery api cost')
                break
    elif r.status_code == 403: # api갱신이 필요
        print('you need api renewal')
        print('break')
        break

    # 위의 예외처리 코드를 거쳐서 내려왔을 때 해당 코드가 실행될 수 있도록 작성

    ## participantIdentities (include player), teams, participants (include runes, stats, timeline, masteries)
    mat = pd.DataFrame(list(r.json().values()), index=list(r.json().keys())).T
    match.append(mat)

print('ALL DONE!')
match = pd.concat(match)

In [ ]:
# 최종 DataFrame 나중 구분을 위한 gameId 칼럼 추가
matchId_df = pd.DataFrame(list(match['gameId']))
matchId_df.columns = ['gameId']

In [ ]:
# <Ingame data> - team

#챔피언아이디를 제외하고 딕셔너리를 뽑는다
a_ls = list(match['teams'])

#team1
team1_df = pd.DataFrame()
for i in range(len(a_ls)):
    try:
        a_ls[i][0].pop('bans',None)
        team1 = pd.DataFrame(list(a_ls[i][0].values()),index = list(a_ls[i][0].keys())).T
        team1_df = team1_df.append(team1)
    except:
        pass
    
team1_df.index = range(len(team1_df))

#team2
team2_df = pd.DataFrame()
for i in range(len(a_ls)):
    try:
        a_ls[i][1].pop('bans',None)
        team2 = pd.DataFrame(list(a_ls[i][1].values()),index = list(a_ls[i][1].keys())).T
        team2_df = team2_df.append(team2)
    except:
        pass
    
team2_df.index = range(len(team2_df))
#컬럼으로 풀어준 team1과 team2와 duration의 데이터를 합쳐준다.
team_df = pd.concat([matchId_df, team1_df,team2_df,match[['gameDuration']].reset_index().drop('index', axis=1)],axis=1)
# print(data_team)


# team_df.to_csv('/content/drive/MyDrive/Colab Notebooks/' + tier + '/' + 
#                 tier + '_' + division + '_03team_data(' + \
#               str(start_point) + '-'+ str(end_point) + ').csv',index=False,encoding = 'cp949') #중간저장
team_df.to_csv('/content/drive/MyDrive/Colab Notebooks/' + tier + '/' + \
                tier + '_03team_data(' + \
              str(start_point) + '-'+ str(end_point) + ').csv',index=False,encoding = 'cp949') #중간저장
print('team data DONE')

team data DONE


In [ ]:
# <Ingame data> - Individual (team당 10개씩)

a_ls = list(match['participants'])

indi_df = pd.DataFrame()
part_df = pd.DataFrame()
stats_df = pd.DataFrame()
time_df = pd.DataFrame()

for i in range(len(a_ls)):
    if i % 10 == 0:
        print(i, '번째 진행중')
    try:
        for j in range(10):
            part = pd.DataFrame(list(a_ls[i][j].values()),index = list(a_ls[i][j].keys())).T

            stats = pd.DataFrame(dict(part['stats'])).T
            # print(stats)
            stats_df = stats_df.append(stats) # [firstInhibitorKill, firstInhibitorAssist]가 있는 data도 있고, 없는 데이터도 있음
            
            stats_df.index = range(len(stats_df))
            # print(stats_df)

            timeline = pd.DataFrame(dict(part['timeline'])).T
            time_df = time_df.append(timeline)
            
            time_df.index = range(len(time_df))
            # print(time_df)


            part = part.drop(['stats', 'timeline'], axis=1)
            part.insert(0, 'matchId', list(matchId_df.values)[i])
            part_df = part_df.append(part)
            # print(part_df)

    except:
        pass
part_df.index = range(len(part_df))


# 이로써 기본 match data와 DTO 칼럼 이였던 participants data (stats, timeline) 병합
indi_df = pd.concat([part_df, stats_df.drop(['participantId'], axis=1), time_df.drop(['participantId'], axis=1)],axis=1)
pd.set_option('display.max_columns', 500) # 펼쳐 보기 
# print(indi_df)

indi_df.to_csv('/content/drive/MyDrive/Colab Notebooks/' + tier + '/' + \
                tier + '_04individual_data(' + \
              str(start_point) + '-'+ str(end_point) + ').csv',index=False,encoding = 'cp949') #중간저장,index=False, encoding = 'cp949') #중간저장

print('ALL DONE, PLEASE CHECK THE CSV FILE IN YOUR DRIVE ACCOUNT')  

0 번째 진행중
10 번째 진행중
20 번째 진행중
30 번째 진행중
40 번째 진행중
50 번째 진행중
60 번째 진행중
70 번째 진행중
80 번째 진행중
90 번째 진행중
100 번째 진행중
110 번째 진행중
120 번째 진행중
130 번째 진행중
140 번째 진행중
150 번째 진행중
160 번째 진행중
170 번째 진행중
180 번째 진행중
190 번째 진행중
200 번째 진행중
210 번째 진행중
220 번째 진행중
230 번째 진행중
240 번째 진행중
250 번째 진행중
260 번째 진행중
270 번째 진행중
280 번째 진행중
290 번째 진행중
300 번째 진행중
310 번째 진행중
320 번째 진행중
330 번째 진행중
340 번째 진행중
350 번째 진행중
360 번째 진행중
370 번째 진행중
380 번째 진행중
390 번째 진행중
400 번째 진행중
410 번째 진행중
420 번째 진행중
430 번째 진행중
440 번째 진행중
450 번째 진행중
460 번째 진행중
470 번째 진행중
480 번째 진행중
490 번째 진행중
500 번째 진행중
510 번째 진행중
520 번째 진행중
530 번째 진행중
540 번째 진행중
550 번째 진행중
560 번째 진행중
570 번째 진행중
580 번째 진행중
590 번째 진행중
600 번째 진행중
610 번째 진행중
620 번째 진행중
630 번째 진행중
640 번째 진행중
650 번째 진행중
660 번째 진행중
670 번째 진행중
680 번째 진행중
690 번째 진행중
700 번째 진행중
710 번째 진행중
720 번째 진행중
730 번째 진행중
740 번째 진행중
750 번째 진행중
760 번째 진행중
770 번째 진행중
780 번째 진행중
790 번째 진행중
800 번째 진행중
810 번째 진행중
820 번째 진행중
830 번째 진행중
840 번째 진행중
850 번째 진행중
860 번째 진행중
870 번째 진행중
880 번째 진행중
890 번째 진행중
900 번째 진행중
910 번째 진행중

In [ ]:
print(indi_df.shape)

(99800, 118)
